In [66]:
import pandas as pd
import igraph as ig 
import seaborn as sns
import numpy as np
import lzma
from Bio import Align

In [67]:
#using readlines to ensure the file is read as a list of lines
with lzma.open('data/sequences.txt.xz', 'rt', encoding='utf-8') as f:
    sequences = f.readlines()

In [68]:
from Bio import Align 
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = Align.substitution_matrices.load("BLOSUM62")
aligner.mode = "global"


In [69]:
nsequence = len(sequences)
similarity_matrix = np.zeros((nsequence, nsequence))

In [70]:
max_scores = np.zeros(nsequence)
for i in range(nsequence):
    seq = sequences[i].strip()
    # S_i,i is the maximum score for sequence i
    max_scores[i] = aligner.score(seq, seq)

In [71]:
sequences[0]

'KRLFDLTLSVLILPFVLPLMGIIALAIKLDSPGPVLYRARRVGENGKLFYMLKFRTMVQDADKRLHEVMQTDEQGNLVYKRPDDPRVTRVGRFLRRTSLDELPQLFNVLRGEMSLVGPRPELPELVERYALWQRKRFAVPQGITGWWQINGRADKPMHLHTEDDLYYVQNYSLWLDLLILWRTFWAVLR\n'

In [72]:
for i in range(nsequence): 
    seq_i_cleaned = str(sequences[i]).strip()
    S_max_i = max_scores[i]
    for j in range(nsequence): 
        seq_j_cleaned = str(sequences[j]).strip()

        score = aligner.score(seq_i_cleaned, seq_j_cleaned)
        similarity_matrix[i, j] = score

In [73]:
similarity_matrix

array([[998., 132., 194., ..., 555.,  95.,  83.],
       [132., 515., 132., ..., 137.,  95., 129.],
       [194., 132., 914., ..., 228., 112.,  95.],
       ...,
       [555., 137., 228., ..., 960., 105., 100.],
       [ 95.,  95., 112., ..., 105., 441.,  90.],
       [ 83., 129.,  95., ..., 100.,  90., 440.]], shape=(513, 513))

In [74]:
diag = np.diag(similarity_matrix)

# 3. Calculate the normalization denominator
# We need a matrix where position (i, j) = sqrt(diag[i] * diag[j])
# The outer product is efficient for creating this grid of values.
denominator = np.sqrt(np.outer(diag, diag))

# 4. Perform element-wise division
normalized_matrix = similarity_matrix / denominator

# # --- Verification ---
# # The diagonal should now be all 1.0s
# print("Diagonal check (should be all 1s):")
# print(np.diag(normalized_matrix)[:5]) 

# # Inspect the result
# print("\nNormalized Matrix (first 3x3):")
# print(np.round(normalized_matrix[:3, :3], 3))

mini=normalized_matrix[:100, :100]

In [75]:
normalized_matrix

array([[1.        , 0.18412173, 0.20312499, ..., 0.5670118 , 0.14319869,
        0.12525252],
       [0.18412173, 1.        , 0.19239652, ..., 0.19484144, 0.19934287,
        0.27099405],
       [0.20312499, 0.19239652, 1.        , ..., 0.24340312, 0.17641099,
        0.14980426],
       ...,
       [0.5670118 , 0.19484144, 0.24340312, ..., 1.        , 0.16137431,
        0.15386436],
       [0.14319869, 0.19934287, 0.17641099, ..., 0.16137431, 1.        ,
        0.20431341],
       [0.12525252, 0.27099405, 0.14980426, ..., 0.15386436, 0.20431341,
        1.        ]], shape=(513, 513))

In [123]:
num_sequences = normalized_matrix.shape[0]
edges = [(i, j) for i in range(num_sequences) for j in range(i + 1, num_sequences) if
            normalized_matrix[i, j] > 0.4]
graph = ig.Graph()
graph.add_vertices(num_sequences)
graph.add_edges(edges)

In [124]:
communities = graph.community_infomap()
print(communities.membership)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 6, 11, 12, 5, 13, 5, 6, 14, 15, 2, 6, 2, 5, 6, 2, 0, 2, 2, 16, 6, 17, 0, 0, 0, 0, 15, 18, 15, 15, 15, 15, 6, 2, 2, 10, 19, 6, 0, 0, 6, 0, 6, 6, 0, 2, 2, 2, 0, 6, 0, 0, 6, 2, 6, 2, 20, 0, 21, 15, 2, 2, 6, 5, 3, 6, 3, 3, 2, 22, 0, 0, 0, 0, 22, 0, 0, 10, 10, 23, 0, 5, 16, 15, 16, 15, 24, 16, 0, 0, 0, 22, 25, 22, 0, 0, 16, 0, 22, 22, 5, 6, 0, 0, 26, 22, 0, 27, 28, 28, 15, 28, 6, 16, 6, 6, 29, 6, 15, 15, 29, 0, 29, 28, 30, 0, 0, 6, 15, 0, 15, 28, 0, 6, 6, 0, 16, 15, 15, 15, 31, 15, 15, 32, 33, 0, 0, 0, 15, 2, 2, 0, 0, 0, 0, 22, 0, 23, 15, 0, 29, 34, 15, 0, 23, 15, 15, 5, 15, 35, 5, 0, 0, 23, 23, 0, 35, 23, 0, 15, 16, 36, 0, 37, 16, 38, 0, 0, 15, 16, 15, 15, 0, 37, 15, 37, 37, 35, 0, 0, 37, 39, 16, 37, 0, 40, 16, 0, 37, 16, 16, 16, 35, 37, 15, 16, 15, 37, 35, 37, 15, 0, 0, 37, 0, 0, 35, 15, 35, 15, 24, 15, 15, 0, 0, 37, 16, 15, 15, 0, 15, 37, 16, 5, 15, 15, 37, 37, 0, 0, 0, 0, 37, 37, 35, 37, 15, 15, 35, 41, 10, 42, 42, 16, 16, 0, 42, 16, 16, 35, 43, 35, 42

In [125]:
import seaborn

rainbow = seaborn.color_palette("husl", len(communities))
colours = [rainbow[communities.membership[v]] for v in graph.vs.indices]
graph.vs["color"] = colours

In [129]:
plot = ig.plot(graph, bbox=(1280, 1280), layout="fr", target = "stable_network_04t.png")

In [127]:
plot